# Portfolio Management. Formulas CFA

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from sklearn import covariance

import datetime as dt
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import random

jtplot.style(figsize=(15, 9))

In [2]:
tickers  = ['AMD', 'AMZN', 'BAC', 'META', 'AVGO', 'TER']
amounts  = [20, 10, 30, 10, 2, 5]

In [3]:
df = yf.download(tickers, dt.datetime(2023, 1, 1), dt.datetime.now(), auto_adjust=True)
df.head()

[*********************100%***********************]  6 of 6 completed


Close                                                \
                  AMD       AMZN        AVGO        BAC        META   
Date                                                                  
2023-01-03  64.019997  85.820000  538.868408  32.075710  124.607788   
2023-01-04  64.660004  85.139999  545.450012  32.678745  127.235008   
2023-01-05  62.330002  83.120003  540.367798  32.611744  126.805458   
2023-01-06  63.959999  86.080002  572.895813  32.937191  129.882202   
2023-01-09  67.239998  87.360001  561.660461  32.439449  129.332779   

                            High                                    ...  \
                  TER        AMD       AMZN        AVGO        BAC  ...   
Date                                                                ...   
2023-01-03  85.381950  66.879997  86.959999  552.586466  32.630887  ...   
2023-01-04  86.862442  65.790001  86.980003  553.160913  33.329640  ...   
2023-01-05  86.564369  64.349998  85.419998  545.060560  32.678748  ...   
2023-01-06  90.658081  64.300003  86.400002  575.397974  33.099917  ...   
2023-01-09  92.615524  69.320000  89.480003  585.786253  33.272211  ...   

                  Open                                      Volume            \
                  AVGO        BAC        META        TER       AMD      AMZN   
Date                                                                           
2023-01-03  550.084305  31.807696  122.689825  88.283325  46851800  76706000   
2023-01-04  548.127414  32.449020  127.244992  86.683590  47477100  68885100   
2023-01-05  541.039585  32.468163  125.996311  86.286149  46159500  67930800   
2023-01-06  548.253983  32.630888  128.833312  87.716960  70161300  83303400   
2023-01-09  576.663620  33.262640  131.020990  92.347250  69741300  65266100   

                                                  
               AVGO       BAC      META      TER  
Date                                              
2023-01-03  2017300  35221500  35528500  1033200  
2023-01-04  1707400  41998500  32397100  2025400  
2023-01-05  1627700  34177000  25447100   998700  
2023-01-06  3526100  34068700  27584500  1444500  
2023-01-09  3773900  43818800  26649100  1654500  

[5 rows x 30 columns]

In [4]:
portfolio = pd.DataFrame(index=df.index, columns=tickers, data=0)
portfolio.head()

,AMD,AMZN,BAC,META,AVGO,TER
Date,,,,,,
2023-01-03,0,0,0,0,0,0
2023-01-04,0,0,0,0,0,0
2023-01-05,0,0,0,0,0,0
2023-01-06,0,0,0,0,0,0
2023-01-09,0,0,0,0,0,0


In [5]:
for ticker, amount in zip(tickers, amounts):
    portfolio[ticker] = df['Close'][ticker] * amount
    portfolio[f'{ticker}_returns'] = np.log(portfolio[ticker].div(portfolio[ticker].shift(1)))

portfolio.dropna(axis=0, inplace=True)
portfolio.head()

,AMD,AMZN,BAC,META,AVGO,TER,AMD_returns,AMZN_returns,BAC_returns,META_returns,AVGO_returns,TER_returns
Date,,,,,,,,,,,,
2023-01-04,1293.200073,851.399994,980.362358,1272.350082,1090.900024,434.312210,0.009947,-0.007955,0.018626,0.020865,0.012140,0.017191
2023-01-05,1246.600037,831.200027,978.352318,1268.054581,1080.735596,432.821846,-0.036700,-0.024012,-0.002052,-0.003382,-0.009361,-0.003437
2023-01-06,1279.199982,860.800018,988.115730,1298.822021,1145.791626,453.290405,0.025815,0.034992,0.009930,0.023974,0.058454,0.046207
2023-01-09,1344.799957,873.600006,973.183479,1293.327789,1123.320923,463.077621,0.050010,0.014760,-0.015227,-0.004239,-0.019806,0.021362
2023-01-10,1361.000061,898.700027,979.788094,1328.490448,1119.504272,470.877571,0.011974,0.028327,0.006764,0.026825,-0.003403,0.016703


## 1. Portfolio Management: an Overview

### Diversification Ratio

In [7]:
equally_weighted_portfolio = (df['Close'] * 1 / len(tickers)).sum(axis=1)
equally_weighted_portfolio_returns = np.log(equally_weighted_portfolio.div(equally_weighted_portfolio.shift(1)))[1:]

random_asset = df['Close'][random.choice(tickers)]
random_asset_returns = np.log(random_asset.div(random_asset.shift(1)))[1:]

diversification_ratio = equally_weighted_portfolio_returns.std() / random_asset_returns.std()
diversification_ratio

0.9257696525320608

In [8]:
returns = [f'{t}_returns' for t in tickers]

diversification_ratio = portfolio[returns].sum(axis=1).std() / portfolio[random.choice(returns)].std()
diversification_ratio

3.112526456404909

### Net Asset Value per Share (NAVPS)

In [9]:
fund_assets = portfolio[tickers].iloc[-1].sum()
fund_liabilities = 0
number_of_shares_outstanding = sum(amounts)
NAV = fund_assets - fund_liabilities

net_asset_value_per_share = NAV / number_of_shares_outstanding
number_of_shares_outstanding

77

## 2. Portfolio Risk and Return: Part I

### Holding Period Return (HPR)

In [16]:
# No cash flows
beginning_value = portfolio.iloc[0][tickers].sum()
ending_value = portfolio.iloc[-1][tickers].sum()

HPR = (ending_value - beginning_value) / beginning_value
HPR

1.5316936072226048

In [47]:
# multiple periods
r_month_periods = portfolio[returns].resample('1M').sum().sum(axis=1)

HPR_multiple_periods = 1
for r in r_month_periods:
    HPR_multiple_periods *= (1 + r)

HPR_multiple_periods -= 1
HPR_multiple_periods

28.838162157358706

### Arithmetic mean return

In [43]:
portfolio[returns].resample('1M').sum()

,AMD_returns,AMZN_returns,BAC_returns,META_returns,AVGO_returns,TER_returns
Date,,,,,,
2023-01-31,0.160291,0.183738,0.057125,0.177513,0.055403,0.168494
2023-02-28,0.044631,-0.090252,-0.033824,0.160698,0.015738,-0.004507
2023-03-31,0.220952,0.091802,-0.175274,0.191860,0.083676,0.061078
2023-04-30,-0.092284,0.020696,0.023498,0.125668,-0.023723,-0.162557
2023-05-31,0.279678,0.134077,-0.052229,0.096701,0.254364,0.093167
2023-06-30,-0.037054,0.077986,0.039820,0.080740,0.076336,0.105430
2023-07-31,0.004292,0.025149,0.109187,0.104524,0.035359,0.014358
2023-08-31,-0.078907,0.031877,-0.101586,-0.073949,0.026616,-0.044905
2023-09-30,-0.027814,-0.082195,-0.046038,0.014494,-0.099954,-0.071167
